In [11]:
from preprocessing.prep import prepare_paths, __create_labels__, get_all_structure, get_all_structure, remover_sublistas_redundantes

In [2]:
import pandas as pd

In [9]:
import os
import json
from tqdm import tqdm

In [4]:

args = pd.Series({
    "root_dir": "/mnt/disks/data/",
    "dataset_path": "/mnt/disks/data/fma/fma_large",
    "embeddings": "music_style",
    "sequence_size": 1280,
    "train_id": "hierarchical_hiclass",
    'sample_size': 1/100
})



In [5]:
args, tracks_df = prepare_paths(args)


In [6]:
tracks_df

,track_genres_all,track_id_,file_path
28793,"[32, 33, 38, 17, 250]",44300,/mnt/disks/data/fma/fma_large/044/044300.mp3
90786,"[38, 12, 47, 1235, 26]",136540,/mnt/disks/data/fma/fma_large/136/136540.mp3
31329,"[362, 10]",48452,/mnt/disks/data/fma/fma_large/048/048452.mp3
97998,"[100, 21]",146243,/mnt/disks/data/fma/fma_large/146/146243.mp3
12027,"[250, 38, 47]",19988,/mnt/disks/data/fma/fma_large/019/019988.mp3
...,...,...,...
50297,"[12, 45, 31]",75222,/mnt/disks/data/fma/fma_large/075/075222.mp3
58005,"[236, 13, 15]",87252,/mnt/disks/data/fma/fma_large/087/087252.mp3
24052,"[538, 18, 1235]",38142,/mnt/disks/data/fma/fma_large/038/038142.mp3
44840,"[224, 38, 15]",67185,/mnt/disks/data/fma/fma_large/067/067185.mp3


In [7]:
def prepare_labels(tracks_df,args):
    ##### Labels
    # Loand genres df
    genres_df = pd.read_csv(os.path.join(args.metadata_path, 'genres.csv'))
    # Mapear os identificadores numéricos de gêneros para os nomes dos gêneros
    genre_names = dict(zip(genres_df['genre_id'], genres_df['title']))

    # Inicialize uma lista para armazenar todos os caminhos de gêneros para cada exemplo
    estruturas = []

    # Iterar sobre as faixas e seus gêneros associados
    for track_genres in tracks_df['track_genres_all']:
        #caminho_name = [genre_names[genre_id] for genre_id in track_genres]
        caminho_id = [get_all_structure(genre_id, genres_df) for genre_id in track_genres]
        estruturas.append(caminho_id)

    for idx, caminho in enumerate(estruturas):
        caminho.sort(key=len, reverse=True)
        estruturas[idx] = remover_sublistas_redundantes(caminho)

    ## Get structure form hierarchical classification
    tracks_df['full_genre_id'] = estruturas
    tracks_df = tracks_df[['track_id_', 'full_genre_id']]

    ## Calculate labels_size
    max_depth = tracks_df.full_genre_id.apply(lambda x: len(x))
    max_depth = int(max_depth.max())
    
    labels_name = []
    for level in range(max_depth):
        labels_name.append(f'label_{level+1}')
        
    tqdm.pandas()
    ## Gnetare categories_df
    #labels =  tracks_df.full_genre_id.progress_apply(lambda x: get_labels_per_level(x))

    #all_levels = categories_df.label5.progress_apply(lambda x: split_label(x))
    all_labels = []
    for idx, row in enumerate(tracks_df.full_genre_id):
        for labels in row:
            labels.extend([0] * (max_depth - len(labels)))
            all_labels.append(labels)
            

    categories_df = pd.DataFrame(all_labels, columns=labels_name)
    
    print(categories_df)
    #categories_df['label_5_name'] = categories_df.level5.apply(lambda x: get_labels_name(x, genres_df))

    # Write labels file
    with open(args.categories_labels_path, 'w+') as f:
        f.write(json.dumps(__create_labels__(categories_df)))

    return tracks_df


In [12]:
tracks_df_new = prepare_labels(tracks_df,args)

      label_1  label_2  label_3  label_4  label_5
0          38       32        0        0        0
1          17       33        0        0        0
2          38       47        0        0        0
3          12       26        0        0        0
4        1235        0        0        0        0
...       ...      ...      ...      ...      ...
1748     1235       18      538        0        0
1749       38      224        0        0        0
1750       15        0        0        0        0
1751       38       32        0        0        0
1752       15      184        0        0        0

[1753 rows x 5 columns]


AttributeError: 'DataFrame' object has no attribute 'level1'

In [13]:
genres_df = pd.read_csv(os.path.join(args.metadata_path, 'genres.csv'))

In [16]:
genres_df[genres_df['parent'] == 0]

,genre_id,#tracks,parent,title,top_level
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
7,8,868,0,Old-Time / Historic,8
8,9,1987,0,Country,9
9,10,13845,0,Pop,10
11,12,32923,0,Rock,12
12,13,730,0,Easy Listening,13
13,14,1499,0,Soul-RnB,14


In [ ]:
#split_dataset(tracks_df,args)